In [1]:
#Import the relevant libraries
import pandas as pd
import numpy as np

In [2]:
#load the data
data_preprocessed = pd.read_csv('Absenteeism_preprocessed.csv')

In [3]:
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


In [4]:
#Creating the targets
data_preprocessed['Absenteeism Time in Hours'].median()

3.0

In [5]:
#our cutoff line will be 3 hours
targets = np.where(data_preprocessed['Absenteeism Time in Hours']>3,1,0)# or targets = np.where(data_preprocessed['Absenteeism Time in Hours']>data_preprocessed['Absenteeism Time in Hours'].median(),1,0)

In [6]:
data_preprocessed['Excessive Absenteeism'] = targets

In [7]:
data_preprocessed.head(10)

,Reason_1,Reason_2,Reason_3,Reason_4,Month value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Excessive Absenteeism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2,0
5,0,0,0,1,7,4,179,51,38,239.554,31,0,0,0,2,0
6,0,0,0,1,7,4,361,52,28,239.554,27,0,1,4,8,1
7,0,0,0,1,7,4,260,50,36,239.554,23,0,4,0,4,1
8,0,0,1,0,7,0,155,12,34,239.554,25,0,2,0,40,1
9,0,0,0,1,7,0,235,11,37,239.554,29,1,1,1,8,1


In [8]:
# Advantage of using a median as the cut off line is that we have divided the data into 50-50(almost)
data_preprocessed['Excessive Absenteeism'].value_counts()

Excessive Absenteeism
0    381
1    319
Name: count, dtype: int64

In [9]:
targets.sum() / targets.shape[0]# A 60-40 also works

0.45571428571428574

In [10]:
# We don't need absenteeism in hours
data_with_targets = data_preprocessed.drop(['Absenteeism Time in Hours','Day of the Week','Distance to Work','Daily Work Load Average',],axis=1)

In [11]:
data_with_targets is data_preprocessed

False

In [12]:
#Declaring Inputs
unscaled_inputs = data_with_targets.iloc[:,:-1]

In [13]:
#Standardizing the data(inputs)
from sklearn.preprocessing import StandardScaler

absenteeism_scaler = StandardScaler()

In [14]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

class CustomScaler(BaseEstimator,TransformerMixin): 
    
    # init or what information we need to declare a CustomScaler object
    # and what is calculated/declared as we do
    
    def __init__(self,columns):
        
        # scaler is nothing but a Standard Scaler object
        self.scaler = StandardScaler()
        # with some columns 'twist'
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        
    
    # the fit method, which, again based on StandardScale
    
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    # the transform method which does the actual scaling

    def transform(self, X, y=None, copy=None):
        
        # record the initial order of the columns
        init_col_order = X.columns
        
        # scale all features that you chose when creating the instance of the class
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        
        # declare a variable containing all information that was not scaled
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        
        # return a data frame which contains all scaled features and all 'not scaled' features
        # use the original order (that you recorded in the beginning)
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [15]:
unscaled_inputs.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month value',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [16]:
columns_to_omit = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4','Education']

In [17]:
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

In [18]:
absenteeism_scaler = CustomScaler(columns_to_scale)

In [19]:
absenteeism_scaler.fit(unscaled_inputs)

C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\numpy\core\fromnumeric.py:3785: FutureWarning: The behavior of DataFrame.var with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return var(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


CustomScaler(columns=['Month value', 'Transportation Expense', 'Age',
                      'Body Mass Index', 'Children', 'Pets'])

In [20]:
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)

In [21]:
scaled_inputs.shape

(700, 11)

In [22]:
scaled_inputs

,Reason_1,Reason_2,Reason_3,Reason_4,Month value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets
0,0,0,0,1,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
1,0,0,0,0,0.182726,-1.574681,2.130803,1.002633,0,-0.019280,-0.589690
2,0,0,0,1,0.182726,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690
3,1,0,0,0,0.182726,0.854936,0.405184,-0.643782,0,0.880469,-0.589690
4,0,0,0,1,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,-0.388293,-0.654143,0.562059,-1.114186,1,0.880469,-0.589690
696,1,0,0,0,-0.388293,0.040034,-1.320435,-0.643782,0,-0.019280,1.126663
697,1,0,0,0,-0.388293,1.624567,-1.320435,-0.408580,1,-0.919030,-0.589690
698,0,0,0,1,-0.388293,0.190942,-0.692937,-0.408580,1,-0.919030,-0.589690


In [23]:
#Now splitting the data into train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs,targets, train_size = 0.9,random_state=20)

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [25]:
#Training the model
reg = LogisticRegression()
reg.fit(x_train, y_train)

LogisticRegression()

In [26]:
reg.score(x_train, y_train)

0.7746031746031746

In [27]:
#Manually checking the accuracy
model_outputs = reg.predict(x_train)

In [28]:
model_outputs == y_train

array([ True,  True,  True,  True,  True,  True,  True, False,  True,
       False,  True,  True,  True, False, False,  True, False, False,
        True,  True,  True,  True, False, False,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True, False,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True, False, False,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True, False, False,  True,  True,  True,  True, False,  True,
       False,  True, False, False,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
       False, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,

In [29]:
np.sum(model_outputs == y_train) / model_outputs.shape[0]

0.7746031746031746

In [30]:
#Now so as to integrate with Tableau, we will define the regression, by finding the coefficients and intercept
feature_name = unscaled_inputs.columns.values

In [31]:
summary_table = pd.DataFrame(columns = ['Feature Name'], data = feature_name)
summary_table['Coefficients'] = np.transpose(reg.coef_)
summary_table

,Feature Namw,Coefficients
0,Reason_1,2.819770
1,Reason_2,0.974119
2,Reason_3,3.167649
3,Reason_4,0.880555
4,Month value,0.135178
5,Transportation Expense,0.607500
6,Age,-0.242874
7,Body Mass Index,0.288012
8,Education,-0.162522
9,Children,0.370068


In [32]:
#Adding intercept
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table

,Feature Namw,Coefficients
0,Intercept,-1.728495
1,Reason_1,2.819770
2,Reason_2,0.974119
3,Reason_3,3.167649
4,Reason_4,0.880555
5,Month value,0.135178
6,Transportation Expense,0.607500
7,Age,-0.242874
8,Body Mass Index,0.288012
9,Education,-0.162522


In [33]:
#Some the weights could be useless
summary_table['Odds ratio'] = np.exp(summary_table.Coefficients)


In [34]:
summary_table.sort_values('Odds ratio',ascending=False)

,Feature Namw,Coefficients,Odds ratio
3,Reason_3,3.167649,23.751588
1,Reason_1,2.819770,16.772985
2,Reason_2,0.974119,2.648832
4,Reason_4,0.880555,2.412238
6,Transportation Expense,0.607500,1.835836
10,Children,0.370068,1.447833
8,Body Mass Index,0.288012,1.333773
5,Month value,0.135178,1.144740
9,Education,-0.162522,0.849997
7,Age,-0.242874,0.784370


In [35]:
### Note: We rescaled the inputs only omitting reasons and education because of their dummy nature, so there might be some 
### confusion on flow of this notebook
# We also dropped the coefficients, which were insignificant
# Testing the model
reg.score(x_test,y_test)

0.7428571428571429

In [36]:
predicted_proba = reg.predict_proba(x_test)

In [37]:
predicted_proba# We are seeing probability of 0 or 1 being the target

array([[0.72826673, 0.27173327],
       [0.61430032, 0.38569968],
       [0.46401386, 0.53598614],
       [0.78492534, 0.21507466],
       [0.082586  , 0.917414  ],
       [0.3442049 , 0.6557951 ],
       [0.3139568 , 0.6860432 ],
       [0.1449294 , 0.8550706 ],
       [0.78061731, 0.21938269],
       [0.75772093, 0.24227907],
       [0.51735876, 0.48264124],
       [0.20668709, 0.79331291],
       [0.06661956, 0.93338044],
       [0.72862101, 0.27137899],
       [0.3696634 , 0.6303366 ],
       [0.51490877, 0.48509123],
       [0.56523503, 0.43476497],
       [0.55572787, 0.44427213],
       [0.47287864, 0.52712136],
       [0.05429509, 0.94570491],
       [0.70750999, 0.29249001],
       [0.78492534, 0.21507466],
       [0.44299417, 0.55700583],
       [0.44299417, 0.55700583],
       [0.24349533, 0.75650467],
       [0.74579208, 0.25420792],
       [0.51744521, 0.48255479],
       [0.8498778 , 0.1501222 ],
       [0.21395505, 0.78604495],
       [0.78492534, 0.21507466],
       [0.

In [38]:
predicted_proba.shape

(70, 2)

In [39]:
predicted_proba[:,1]

array([0.27173327, 0.38569968, 0.53598614, 0.21507466, 0.917414  ,
       0.6557951 , 0.6860432 , 0.8550706 , 0.21938269, 0.24227907,
       0.48264124, 0.79331291, 0.93338044, 0.27137899, 0.6303366 ,
       0.48509123, 0.43476497, 0.44427213, 0.52712136, 0.94570491,
       0.29249001, 0.21507466, 0.55700583, 0.55700583, 0.75650467,
       0.25420792, 0.48255479, 0.1501222 , 0.78604495, 0.21507466,
       0.34982618, 0.67855443, 0.67300338, 0.50193156, 0.21507466,
       0.54598373, 0.2260637 , 0.75160167, 0.37468797, 0.61502963,
       0.20863087, 0.47422655, 0.23985337, 0.39842047, 0.78310198,
       0.56663671, 0.69772383, 0.27173327, 0.20806972, 0.20233038,
       0.57965457, 0.33848669, 0.6557951 , 0.24879815, 0.81363273,
       0.40414211, 0.87932608, 0.24207681, 0.30713447, 0.31540807,
       0.70988511, 0.64703161, 0.29501202, 0.79027503, 0.20578823,
       0.25672976, 0.0852201 , 0.2260637 , 0.72121458, 0.30295643])

In [40]:
#Saving the model through picle
import pickle

In [41]:
#with open('model','wb') as file:
    #pickle.dump(reg, file)

In [42]:
#with open('scaler', 'wb') as file:
#    pickle.dump(absenteeism_scaler, file)